In [4]:
import numpy as np
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

Dataset : Employee data used in lecture linked here.\
1.Preprocess Test data and get predictions \
2.Compute Mean Absolute Error, Mean Square error for test data. \
3.Optional : Read about Ridge and Lasso Regression, implement them and compute evaluation metrics. Do they perform better than Linear Regression?

In [20]:
data = pd.read_csv("https://raw.githubusercontent.com/abirami1998/NYU-Data-Science-Bootcamp-Spring-2024/main/Week%206/employee.csv")
data = data.drop(columns=['id', 'timestamp','country'])

In [21]:
data.loc[data['hours_per_week'].isna(), 'hours_per_week'] = data['hours_per_week'].median()
data.loc[data['telecommute_days_per_week'].isna(), 'telecommute_days_per_week'] = data['telecommute_days_per_week'].median()
data = data.dropna()
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 4261 entries, 0 to 4276
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   employment_status              4261 non-null   object 
 1   job_title                      4261 non-null   object 
 2   job_years                      4261 non-null   float64
 3   is_manager                     4261 non-null   object 
 4   hours_per_week                 4261 non-null   float64
 5   telecommute_days_per_week      4261 non-null   float64
 6   education                      4261 non-null   object 
 7   is_education_computer_related  4261 non-null   object 
 8   certifications                 4261 non-null   object 
 9   salary                         4261 non-null   float64
dtypes: float64(4), object(6)
memory usage: 366.2+ KB
None


In [25]:
data_train = data.copy()
data_train.head()

cat_cols = [c for c in data_train.columns if data_train[c].dtype == 'object' and c not in ['is_manager', 'certifications']]
cat_data = data_train[cat_cols]

binary_cols = ['is_manager', 'certifications']
for c in binary_cols:
    data_train[c] = data_train[c].replace(to_replace=['Yes'], value=1)
    data_train[c] = data_train[c].replace(to_replace=['No'], value=0)

final_data = pd.get_dummies(data_train, columns=cat_cols, drop_first= True)
final_data.shape

(4261, 25)

In [28]:
y = final_data['salary']
X = final_data.drop(columns=['salary'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("Training Set Dimensions:", X_train.shape)
print("Validation Set Dimensions:", X_test.shape)

num_cols = ['job_years','hours_per_week','telecommute_days_per_week']

scaler = StandardScaler()
scaler.fit(X_train[num_cols])
X_train[num_cols] = scaler.transform(X_train[num_cols])

X_train

Training Set Dimensions: (3408, 24)
Validation Set Dimensions: (853, 24)


,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,...,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
2727,-0.738314,1,2.379612,-0.629007,1,False,False,False,False,False,...,True,False,False,False,True,False,False,False,False,True
2677,0.267819,0,-0.478071,-0.022941,1,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,True
4051,0.267819,1,-0.478071,-0.022941,1,True,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
1379,1.777018,0,0.665002,-0.629007,1,True,False,False,False,True,...,False,False,False,False,True,False,False,False,False,True
3997,-0.235248,1,-2.192681,-0.629007,0,False,False,False,False,True,...,False,False,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4168,0.267819,0,-0.478071,-0.629007,0,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,True
351,-1.744447,0,-0.478071,-0.629007,0,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
755,-0.068991,0,0.078268,-0.629007,1,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,False
3371,-0.005622,0,0.032933,-0.629007,0,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True


In [29]:
reg=LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [30]:
X_test[num_cols] = scaler.transform(X_test[num_cols])
y_pred = reg.predict(X_test)
y_pred

array([7363.63570888, 6974.10560284, 7322.98994604, 6716.59588392,
       6876.049479  , 6240.39215238, 7764.3788306 , 6971.85261193,
       7081.85381453, 6225.42048606, 6748.22763122, 7106.42336908,
       7574.4827715 , 6925.32441505, 6847.42693198, 6402.71545337,
       6603.83356792, 6922.15658091, 6736.55560463, 7075.63440385,
       7280.12792715, 7585.80100135, 7050.24170379, 6416.95206406,
       7240.68044298, 6665.00195972, 7233.35644002, 5935.76498436,
       7147.40833754, 7593.62052754, 7671.22859283, 6890.61948582,
       7029.46896027, 7109.83708134, 7570.31270621, 6422.46038258,
       6733.83364106, 5935.76498436, 5969.51862189, 6024.7872137 ,
       6369.87170571, 7029.37398942, 6286.86644176, 6824.03655702,
       7830.52992552, 7206.92038726, 7035.39758865, 6648.55354759,
       6998.8865981 , 7068.13515904, 6675.44046256, 7307.66317692,
       7949.64697775, 7809.21393843, 7067.82022003, 6890.61948582,
       6756.08656589, 8115.63363173, 6625.95110079, 7095.86903

In [31]:
mean_absolute_error(y_test,y_pred)

863.9941648588874

In [32]:
mean_squared_error(y_test,y_pred)**0.5

1224.3125551023209